<a href="https://colab.research.google.com/github/MPGarg/EVASub/blob/Session-3/EVA8_Session_3_Assignment_Experiments_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [50]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=0) 
        self.bn1 = nn.BatchNorm2d(8)
        self.drop1 = nn.Dropout(0.05)
        
        self.conv2 = nn.Conv2d(8, 16, 3, padding=0)
        self.bn2 = nn.BatchNorm2d(16)
        self.drop2 = nn.Dropout(0.05)
        
        self.conv3 = nn.Conv2d(16, 16, 3, padding=0)
        self.bn3 = nn.BatchNorm2d(16)
        self.drop3 = nn.Dropout(0.05)
        
        self.conv4 = nn.Conv2d(16, 8, 1, padding=0)
        self.bn4 = nn.BatchNorm2d(8)
        self.pool4 = nn.MaxPool2d(2, 2)
        self.drop4 = nn.Dropout(0.05)
        
        self.conv5 = nn.Conv2d(8, 16, 3, padding=0)
        self.bn5 = nn.BatchNorm2d(16)
        self.drop5 = nn.Dropout(0.05)
        
        self.conv6 = nn.Conv2d(16, 16, 3, padding=0)
        self.bn6 = nn.BatchNorm2d(16)
        self.drop6 = nn.Dropout(0.05)
        
        self.conv6_1 = nn.Conv2d(16, 32, 3, padding=0)
        self.bn6_1 = nn.BatchNorm2d(32)
        self.drop6_1 = nn.Dropout(0.05)
        
        self.conv7 = nn.Conv2d(32, 16, 1, padding=0)
        self.bn7 = nn.BatchNorm2d(16)
        self.pool7 = nn.AvgPool2d(5,5)
        
        self.fc1 = nn.Linear(16, 10)
        
    def forward(self, x):
        x = self.drop1(self.bn1(F.relu(self.conv1(x))))
        x = self.drop2(self.bn2(F.relu(self.conv2(x))))
        x = self.drop3(self.bn3(F.relu(self.conv3(x))))
        x = self.drop4(self.pool4(self.bn4(self.conv4(x))))
        x = self.drop5(self.bn5(F.relu(self.conv5(x))))
        x = self.drop6(self.bn6(F.relu(self.conv6(x))))
        x = self.drop6_1(self.bn6_1(F.relu(self.conv6_1(x))))
        x = self.pool7(self.bn7(self.conv7(x)))
        
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return F.log_softmax(x)

In [51]:
#!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
       BatchNorm2d-2            [-1, 8, 26, 26]              16
           Dropout-3            [-1, 8, 26, 26]               0
            Conv2d-4           [-1, 16, 24, 24]           1,168
       BatchNorm2d-5           [-1, 16, 24, 24]              32
           Dropout-6           [-1, 16, 24, 24]               0
            Conv2d-7           [-1, 16, 22, 22]           2,320
       BatchNorm2d-8           [-1, 16, 22, 22]              32
           Dropout-9           [-1, 16, 22, 22]               0
           Conv2d-10            [-1, 8, 22, 22]             136
      BatchNorm2d-11            [-1, 8, 22, 22]              16
        MaxPool2d-12            [-1, 8, 11, 11]               0
          Dropout-13            [-1, 8, 11, 11]               0
           Conv2d-14             [-1, 1

<ipython-input-50-d9042a50c058>:51: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [52]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [53]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'epoch={epoch} loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [54]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-50-d9042a50c058>:51: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
epoch=1 loss=0.09803862124681473 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 30.57it/s]



Test set: Average loss: 0.0993, Accuracy: 9713/10000 (97.13%)



epoch=2 loss=0.04797786474227905 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 30.55it/s]



Test set: Average loss: 0.0500, Accuracy: 9850/10000 (98.50%)



epoch=3 loss=0.10213446617126465 batch_id=468: 100%|██████████| 469/469 [00:14<00:00, 31.43it/s]


...


Test set: Average loss: 0.0214, Accuracy: 9930/10000 (99.30%)



epoch=18 loss=0.01795389875769615 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 30.58it/s]



Test set: Average loss: 0.0185, Accuracy: 9943/10000 (99.43%)



epoch=19 loss=0.004879422951489687 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 30.74it/s]



Test set: Average loss: 0.0202, Accuracy: 9941/10000 (99.41%)

